# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [17]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [18]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,72.88,83,35,19.86,PN,1732670270
1,1,lihue,21.9789,-159.3672,80.74,65,0,17.27,US,1732670272
2,2,deqen,29.9618,90.7188,22.01,57,0,4.81,CN,1732670274
3,3,jalai nur,49.4500,117.7000,13.73,76,100,18.66,CN,1732670276
4,4,nema,57.5067,50.5011,24.78,88,5,4.00,RU,1732670278


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [19]:
#%%capture --no-display

# Configure the map plot
humidityplot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.7,
    tiles = "OSM",
    frame_height = 504,
    frame_width = 896,
    hover_cols = ["City", "Humidity"]
)
# Display the map
humidityplot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [20]:
# Narrow down cities that fit criteria and drop any results with null values
idealCityDF = city_data_df.loc[
    (city_data_df["Max Temp"] > 69.8) & (city_data_df["Max Temp"] < 80.6) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]

idealCityDf = idealCityDF.dropna()

print(len(idealCityDF))
idealCityDF.head()

8


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
26,26,san ignacio,17.1561,-89.0714,77.14,88,0,0.00,BZ,1732670327
65,65,turbat,26.0023,63.0440,71.71,52,0,2.24,PK,1732670409
111,111,tres palos,18.1833,-101.6167,70.63,96,0,3.33,MX,1732670510
221,221,bafoulabe,13.8065,-10.8321,76.59,17,0,3.11,ML,1732670745
333,333,remire-montjoly,4.9167,-52.2667,78.84,83,0,2.30,GF,1732670917


### Step 3: Create a new DataFrame called `hotel_df`.

In [21]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = idealCityDf[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
26,san ignacio,BZ,17.1561,-89.0714,88,
65,turbat,PK,26.0023,63.0440,52,
111,tres palos,MX,18.1833,-101.6167,96,
221,bafoulabe,ML,13.8065,-10.8321,17,
333,remire-montjoly,GF,4.9167,-52.2667,83,
436,brisas de zicatela,MX,15.8369,-97.0419,89,
506,dwarka,IN,22.2394,68.9678,37,
509,lazaro cardenas,MX,17.9583,-102.2000,75,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [22]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accomidation.hotel",
    "apiKey": geoapify_key,
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?PARAMS"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != "No hotel found"]
hotel_df

Starting hotel search
san ignacio - nearest hotel: No hotel found
turbat - nearest hotel: No hotel found
tres palos - nearest hotel: No hotel found
bafoulabe - nearest hotel: No hotel found
remire-montjoly - nearest hotel: No hotel found
brisas de zicatela - nearest hotel: No hotel found
dwarka - nearest hotel: No hotel found
lazaro cardenas - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

In [ ]:
#wasnt able to get a hotel